In [ ]:
#reload source files automatically
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import graphviz

from event_clustering.preprocessing import *
from event_clustering.clustering import *
from event_clustering.postprocessing import *
from event_clustering.process_mining import *

from sklearn.cluster import *
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import sklearn.metrics as metrics
import sklearn.metrics.cluster as cluster_metrics

import scipy.cluster.hierarchy as sch

# Load Encoded Events

In [ ]:
file_name = 'DomesticDeclarations'
df = preprocess(load('data/' + file_name + '.xes'))
df_encoded = pd.read_csv('data/' + file_name + '_name-role-pos-ref-tod_encoded.csv')
original_df_columns = df.columns

# Clustering

### KMeans

In [ ]:
find_optimal_clusters(df_encoded, 10)

In [ ]:
cluster_nr = 6
km = MiniBatchKMeans(n_clusters=cluster_nr, init_size=1024, batch_size=2048, random_state=20)
cluster_labels = km.fit_predict(df_encoded)

### Hierarchical Clustering

In [ ]:
# setting distance_threshold=0 ensures we compute the full tree.
hier_dendogram = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "single").fit(df_encoded)

# Plot the corresponding dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plot_dendrogram(hier_dendogram, truncate_mode='level', p=5)
plt.xlabel("Sample Index or (Cluster Size)")
plt.ylabel("Distance")
plt.show()

plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plot_dendrogram(hier_dendogram, truncate_mode='level', p=40)
plt.xlabel("Sample Index or (Cluster Size)")
plt.ylabel("Distance")
plt.show()

In [ ]:
# --> determine amount of clusters
agg_clusters = 6
hier = AgglomerativeClustering(n_clusters=agg_clusters, linkage="single").fit(df_encoded)
cluster_labels = hier.labels_

## Evaluation

In [ ]:
plot_pca(df_encoded, cluster_labels)

In [ ]:
print("Completeness KMeans: ", metrics.completeness_score(df['concept:name'], cluster_labels))
print("Silhouette Coefficient KMeans: ", metrics.silhouette_score(df_encoded, cluster_labels))

In [ ]:
df['cluster_label'] = cluster_labels

## Check cluster labels "manually"

In [ ]:
df

In [ ]:
# group by cluster label and aggregate concept:name
eval_manually = df.groupby('cluster_label')['concept:name'].apply(set)

In [ ]:
pd.set_option('display.max_colwidth', -1)
eval_manually

In [ ]:
df_encoded

## Use Supervised Approach

In [ ]:
X = df_encoded
Y = df["cluster_label"]

clf = tree.DecisionTreeClassifier()
clf1 = clf.fit(X,Y)

dot_data = tree.export_graphviz(clf1, out_file=None, feature_names=X.columns, class_names=True, filled=True, rounded=True)
graph = graphviz.Source(dot_data)
graph.render("data/domesticDeclarations_name-pos-time")
#graph

In [ ]:
# split into test and training data set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) #, random_state=42)

In [ ]:
SVM_model = svm.SVC()
SVM_pred = SVM_model.fit(x_train, y_train).predict(x_test)

tree_model = tree.DecisionTreeClassifier()
tree_pred = tree_model.fit(x_train, y_train).predict(x_test)

KNN_model = KNeighborsClassifier(n_neighbors=5)
KNN_pred = KNN_model.fit(x_train, y_train).predict(x_test)

log_model = LogisticRegression()
log_pred = log_model.fit(x_train, y_train).predict(x_test)

gauss_model = GaussianNB()
gauss_pred = gauss_model.fit(x_train, y_train).predict(x_test)

In [ ]:
print("Accuracy Scores\n")
print("SVM Classifier: ", accuracy_score(SVM_pred, y_test))
print("KNN Classifier: ", accuracy_score(KNN_pred, y_test))
print("Decision Tree Classifier: ", accuracy_score(tree_pred, y_test))
print("Logistic Regression Classifier: ", accuracy_score(log_pred, y_test))
print("Gaussian Naive Bayes Classifier: ", accuracy_score(gauss_pred, y_test))

In [ ]:
print("SVM Classifier: ", confusion_matrix(SVM_pred, y_test))
print("KNN Classifier: ", confusion_matrix(KNN_pred, y_test))
print("Decision Tree Classifier: ", confusion_matrix(tree_pred, y_test))
print("Logistic Regression Classifier: ", confusion_matrix(log_pred, y_test))
print("Gaussian Naive Bayes Classifier: ", confusion_matrix(gauss_pred, y_test))

In [ ]:
print("SVM Classifier: ", classification_report(SVM_pred, y_test))
print("KNN Classifier: ", classification_report(KNN_pred, y_test))
print("Decision Tree Classifier: ", classification_report(tree_pred, y_test))
print("Logistic Regression Classifier: ", classification_report(log_pred, y_test))
print("Gaussian Naive Bayes Classifier: ", classification_report(gauss_pred, y_test))

In [ ]:
all_pred = [SVM_pred, tree_pred, KNN_pred, log_pred, gauss_pred]

for i in all_pred:
    print("Number of mislabeled points out of a total %d points : %d" % (x_test.shape[0], (y_test != i).sum()))

In [ ]:
# plot predictions against the true values
plt.scatter(y_test, gauss_pred)
plt.xlabel("True Values")
plt.ylabel("Predictions")
plt.show()

## Replace events with cluster representative

In [ ]:
# overwrite concept:name with mapped cluster representative
abstracted_df = replace_with_representative(df, "concept:name", "cluster_label", original_df_columns)
abstracted_df.head(5)

In [ ]:
# write abstracted dataframe as csv file
csv_filename = "data/dd_7_h.csv"
write_to_csv(abstracted_df, csv_filename, index=False)

# Compare Process Models from Original Dataset to Abstracted Dataset

In [ ]:
# read original and abstracted dataset as event logs
filepath_original = "data/DomesticDeclarations.xes"
filepath_abstracted = csv_filename
original_log = read_as_log_xes(filepath_original)
abstracted_log = read_as_log_csv(filepath_abstracted)

print("Original Log:")
print(original_log)
print("\nAbstracted Log:")
print(abstracted_log)

## Mining Algorithms
* Alpha Miner
* Inductive Miner
* Heuristic Miner
* Directly-Follows Graph

In [ ]:
# Inductive Miner
net_ind_orig, im_ind_orig, fm_ind_orig = ind_miner(original_log)
net_ind, im_ind, fm_ind = ind_miner(abstracted_log)

In [ ]:
# Heuristic Miner
net_heu_orig, im_heu_orig, fm_heu_orig = heu_miner(original_log)
net_heu, im_heu, fm_heu = heu_miner(abstracted_log)

In [ ]:
# Alpha Miner
net_alpha_orig, im_alpha_orig, fm_alpha_orig = a_miner(original_log)
net_alpha, im_alpha, fm_alpha = a_miner(abstracted_log)

In [ ]:
# Directly-Follows Graph
dfg_miner(original_log)
dfg_miner(abstracted_log)

## Process Model Metrics
* Fitness
* Precision
* Generalization
* Simplicity

In [ ]:
# Evaluation --> all four metrices in one line
print("Original Log with Heuristic Miner:")
evaluation_metric(original_log, net_heu_orig, im_heu_orig, fm_heu_orig)

print("\nAbstracted Log with Heuristic Miner")
evaluation_metric(abstracted_log, net_heu, im_heu, fm_heu)

In [ ]:
print("Original Log with Inductive Miner:")
evaluation_metric(original_log, net_ind_orig, im_ind_orig, fm_ind_orig)

print("\nAbstracted Log with Inductive Miner")
evaluation_metric(abstracted_log, net_ind, im_ind, fm_ind)

In [ ]:
print("Original Log with Inductive Miner:")
evaluation_metric(original_log, net_ind_orig, im_ind_orig, fm_ind_orig )

print("\nAbstracted Log with Inductive Miner")
evaluation_metric(abstracted_log, net_ind, im_ind, fm_ind)

In [ ]:
# Fitness
print("Original Log with Heuristic Miner:")
fitness_metric(original_log, net_heu_orig, im_heu_orig, fm_heu_orig)

print("\nAbstracted Log with Heuristic Miner")
fitness_metric(abstracted_log, net_heu, im_heu, fm_heu)

In [ ]:
# Precision
print("Original Log with Heuristic Miner:")
precision_metric(original_log, net_heu_orig, im_heu_orig, fm_heu_orig)

print("\nAbstracted Log with Heuristic Miner")
precision_metric(abstracted_log, net_heu, im_heu, fm_heu)

In [ ]:
# Generalization
print("Original Log with Heuristic Miner:")
generalization_metric(original_log, net_heu_orig, im_heu_orig, fm_heu_orig)

print("\nAbstracted Log with Heuristic Miner")
generalization_metric(abstracted_log, net_heu, im_heu, fm_heu)

In [ ]:
# Simplicity
print("Original Log with Heuristic Miner:")
simplicity_metric(original_log, net_heu_orig, im_heu_orig, fm_heu_orig)

print("\nAbstracted Log with Heuristic Miner")
simplicity_metric(abstracted_log, net_heu, im_heu, fm_heu)